<a href="https://colab.research.google.com/github/8Chatea8/HeadChecker/blob/main/data/data_get.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import zipfile
import os
import glob

In [ ]:
zip_files = glob.glob('/content/drive/MyDrive/Aiffel-MahiMahi/data/Validation/*_EC.zip')
len(zip_files) # 6

6

In [ ]:
part1_zip_files = []
part2_zip_files = []

for f in zip_files:
    if 'Part1' in f:
        part1_zip_files.append(f)
    elif 'Part2' in f:
        part2_zip_files.append(f)
    else:
        print('something else wrong')

print(part1_zip_files, '\n', len(part1_zip_files)) # 3
print(part2_zip_files, '\n', len(part2_zip_files)) # 3

['/content/drive/MyDrive/Aiffel-MahiMahi/data/Validation/VL_Part1_Clickbait_Direct_EC.zip', '/content/drive/MyDrive/Aiffel-MahiMahi/data/Validation/VL_Part1_Clickbait_Auto_EC.zip', '/content/drive/MyDrive/Aiffel-MahiMahi/data/Validation/VL_Part1_NonClickbait_Auto_EC.zip'] 
 3
['/content/drive/MyDrive/Aiffel-MahiMahi/data/Validation/VL_Part2_Clickbait_Direct_EC.zip', '/content/drive/MyDrive/Aiffel-MahiMahi/data/Validation/VL_Part2_NonClickbait_Auto_EC.zip', '/content/drive/MyDrive/Aiffel-MahiMahi/data/Validation/VL_Part2_Clickbait_Auto_EC.zip'] 
 3


In [ ]:
for zip in part1_zip_files:
    zip_file = zipfile.ZipFile(zip)
    zip_file.extractall('/content/sample_data/sample1/')

In [ ]:
data_files = glob.glob('/content/sample_data/sample1/*.json')
len(data_files)

5257

In [ ]:
data = pd.DataFrame()
for files in data_files:
    df = pd.read_json(files)
    df = df.transpose()
    df = df[['newTitle', 'newsContent', 'clickbaitClass']]
    df.iloc[0, 0] = df.iloc[1, 0]
    df.iloc[0, 2] = df.iloc[1, 2]
    df = df.drop('labeledDataInfo', axis=0)
    data = pd.concat([data, df])
data = data.reset_index(drop=True)
data.columns = ['Headline', 'Content', 'Class']

In [ ]:
len(data_files) == len(data)

True

In [ ]:
for zip in part2_zip_files:
    zip_file = zipfile.ZipFile(zip)
    zip_file.extractall('/content/sample_data/sample2/')

In [ ]:
data_files = glob.glob('/content/sample_data/sample2/*.json')
len(data_files)

5133

In [ ]:
data2 = pd.DataFrame()
for files in data_files:
    df = pd.read_json(files)
    df = df.transpose()
    df = df[['newsTitle', 'newsContent', 'clickbaitClass']]
    df.iloc[0, 2] = df.iloc[1, 2]
    df = df.drop('labeledDataInfo', axis=0)
    data2 = pd.concat([data2, df])
data2 = data2.reset_index(drop=True)
data2.columns = ['Headline', 'Content', 'Class']

In [ ]:
len(data_files) == len(data2) # True

True

In [ ]:
df = pd.concat([data, data2])

len(df) == len(data) + len(data2) # True

True

In [ ]:
df.shape

(10390, 3)

In [ ]:
df.isnull().sum()

Headline    0
Content     0
Class       0
dtype: int64

In [ ]:
import re

def preprocess_sentence(sentence):
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거 Ex) my husband (and myself!) for => my husband for
    sentence = re.sub(r'\[[^]]*\]', '', sentence) # [] 닫힌 문자열 제거
    sentence = re.sub(r'\\"','', sentence) # 쌍따옴표 \" 제거
    sentence = re.sub(r'\'', '', sentence) # 따옴표 ' 제거
    return sentence

In [ ]:
df['cleanHeadline'] = df['Headline'].apply(lambda x: preprocess_sentence(x))
df['cleanContent'] = df['Content'].apply(lambda x: preprocess_sentence(x))

In [ ]:
df['HeadLength'] = df['Headline'].apply(lambda x: len(x))
df['ContLength'] = df['Content'].apply(lambda x: len(x))

In [ ]:
df['cleanHeadLength'] = df['cleanHeadline'].apply(lambda x: len(x))
df['cleanContLength'] = df['cleanContent'].apply(lambda x: len(x))

In [ ]:
over_head = 0
threshold_head = 35

over_head = (df['cleanHeadLength'] > threshold_head).sum()

print(f"{threshold_head} 글자가 넘는 제목의 개수: {over_head}")
print(f"{threshold_head} 글자가 넘는 제목이 전체에서 차지하는 비율: {over_head/len(df)*100:.2f}%")

35 글자가 넘는 제목의 개수: 1815
35 글자가 넘는 제목이 전체에서 차지하는 비율: 17.47%


In [ ]:
over_content = 0
threshold_content = 1200

over_content = (df['cleanContLength'] > threshold_content).sum()

print(f"{threshold_content} 글자가 넘는 제목의 개수: {over_content}")
print(f"{threshold_content} 글자가 넘는 제목이 전체에서 차지하는 비율: {over_content/len(df)*100:.2f}%")

1200 글자가 넘는 제목의 개수: 3359
1200 글자가 넘는 제목이 전체에서 차지하는 비율: 32.33%


In [ ]:
original_length = len(df)

over_head_idx = df[df['cleanHeadLength'] > threshold_head].index
df2 = df.drop(over_head_idx)

over_content_idx = df[df['cleanContLength'] > threshold_content].index
df2 = df.drop(over_content_idx)

df2.reset_index(drop=True)

deleted_length = len(df2)

print(f"기존 데이터의 수: {original_length}")
print(f"긴 데이터를 삭제하고 난 이후 데이터의 수: {deleted_length}")

기존 데이터의 수: 10390
긴 데이터를 삭제하고 난 이후 데이터의 수: 4800


In [ ]:
sample_df = df[['cleanHeadline', 'cleanContent', 'Class']].copy()
sample_df.columns = ['Headline', 'Content', 'Class']
sample_df.head()

,Headline,Content,Class
0,경기 악화로 부실 대출채권이 크게 늘자 대출 회수작업 무슨 속셈 인가?,저축은행 업계가 코로나19 사태 이후 대대적으로 대출 회수작업에 나선 것으로 파악됐...,0
1,"똑똑한 소비자, 한여름에 겨울점퍼, 모피를 산다?",요즘 창업시장의 화두는 초기 창업비용 절감과 인건비 절약이다.\n이에 인건비 부담이...,0
2,세종시 땅값 13개월 째 뛰었다,"불공정행위 감소… 악의적 신고 예방책 마련 필요 이동전화 불공정행위 신고포상제, 일...",0
3,서울친환경유통센터 학교 관계자와 시민들과 SNS 소통 확대,서울시농수산식품공사가 운영하는 서울친환경유통센터는 서울시 학교급식에 관한 정보를 적...,1
4,"대전노동청, 충청지역 사회적기업 성공사례 발굴대회",코로나19 사태 이후 우리는 불확실성 시대에 살아가기를 강요받고 있다.\n불안한 미...,0


In [ ]:
sample_df.to_csv('VL_data_EC.csv', index=False, encoding='utf-8')